In [2]:
#import org.apache.spark.sql.SparkSession
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import PCA
from pyspark import *
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

from pyspark.ml.feature import RFormula


In [3]:
spark = SparkSession \
    .builder \
     .appName("Python Spark SQL basic example") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()
 

## Charger les données 

In [4]:
data = spark.read.load('iris.data',
                     format="csv", sep=",", inferSchema="false", header="false")

In [5]:
data.show(10)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
+---+---+---+---+-----------+
only showing top 10 rows



In [6]:
data.columns[0:4]

['_c0', '_c1', '_c2', '_c3']

In [7]:
data = data.withColumnRenamed('_c0', 'Sepal_length').withColumnRenamed('_c1', 'Sepal_width').withColumnRenamed('_c2', 'petal_length').withColumnRenamed('_c3', 'petal_width')

In [8]:
data.show(5)

+------------+-----------+------------+-----------+-----------+
|Sepal_length|Sepal_width|petal_length|petal_width|        _c4|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



## Subdiviser les données en 2 parties 
- les 4 premières colonnes vont servir à faire l'ACP 
- Tandis que la dernière colonne peut servir pour une éventuelle prédiction 

In [9]:
df_pca = data.select([c for c in data.columns if c in data.columns[0:4]])
df_rest = data.select(['_c4'])

In [80]:
#table_pca = df_pca.toDF('_c0','_c1', '_c2', '_c3')

In [87]:
type(table_pca),type(df_pca)

(pyspark.sql.dataframe.DataFrame, pyspark.sql.dataframe.DataFrame)

## Appliquer L'ACP

In [10]:
data_num = RFormula(formula=" ~ {0}".format(" + ".join(df_pca.columns))).fit(df_pca).transform(df_pca)

In [12]:
data_num.show(5)

+------------+-----------+------------+-----------+--------------------+
|Sepal_length|Sepal_width|petal_length|petal_width|            features|
+------------+-----------+------------+-----------+--------------------+
|         5.1|        3.5|         1.4|        0.2|(119,[2,43,57,98]...|
|         4.9|        3.0|         1.4|        0.2|(119,[10,34,57,98...|
|         4.7|        3.2|         1.3|        0.2|(119,[25,36,61,98...|
|         4.6|        3.1|         1.5|        0.2|(119,[17,37,56,98...|
|         5.0|        3.6|         1.4|        0.2|(119,[0,49,57,98]...|
+------------+-----------+------------+-----------+--------------------+
only showing top 5 rows



In [27]:
dataPCA = PCA(k=4, inputCol='features', outputCol="pcaFeatures")
model = dataPCA.fit(data_num)
result = dataPCA.setInputCol("features").fit(data_num).transform(data_num)
result.select("pcaFeatures").show(5,truncate=False)

+---------------------------------------------------------------------------------+
|pcaFeatures                                                                      |
+---------------------------------------------------------------------------------+
|[-1.1675049339011245,0.2001361191815517,-0.3411700830638471,-0.1867667558191073] |
|[-0.7830055063271875,1.091382916447496,-0.14680595434570837,-0.2591421430916775] |
|[-0.936824213265719,0.1557071575852669,-0.5294225908680525,0.6217155249948557]   |
|[-1.031177782899719,-0.07541174652579684,0.7035293357732575,-0.04233672205686603]|
|[-1.1540724414141952,0.29281217297198214,-0.524744666106624,0.005429354440287595]|
+---------------------------------------------------------------------------------+
only showing top 5 rows



In [26]:
model.explainedVariance
model.pc

DenseMatrix(119, 4, [-0.1684, 0.0797, -0.1202, 0.0614, 0.0642, 0.0579, -0.0207, 0.0314, ..., -0.0127, 0.0282, 0.0443, -0.0216, 0.0311, 0.0032, -0.0097, -0.0061], 0)

## Visualisation des résultats du model 

In [28]:
plot(result.select("pcaFeatures"))

NameError: name 'plot' is not defined

## Correlation entre les variables